In [1]:
# Define the parameter
LIF.V_REST = 15*b2.mV
LIF.V_RESET = -50*b2.mV
LIF.FIRING_THRESHOLD = 0*b2.mV
LIF.MEMBRANE_RESISTANCE = 50. * b2.ohm
LIF.MEMBRANE_TIME_SCALE = 10. * b2.ms
ABSOLUTE_REFRACTORY_PERIOD = 0 * b2.ms ####### not sure we have to change this parameter (initially 2.0 * b2.ms) 
beta = 10e5 #s^-1 -> have to precise the factor beta for the instantaneous firing rate

LIF.print_default_parameters()

# Create oscillating external current
Tf = 100 #ms
unit_time = 1*b2.ms
I0 = 0.5*b2.mA
omega = 100*b2.Hz
freq = omega/(2*np.pi)
f_LIF = input_factory.get_sinusoidal_current(0, Tf, unit_time=unit_time, amplitude=I0, frequency=freq, direct_current=I0, append_zero=True)

f_LIF = f_LIF.values[:, 0] / b2.mA
plt.plot(np.linspace(0, Tf+1, Tf+2), f_LIF)
plt.xlabel('Time (ms)')
plt.ylabel('Current (mA)')
plt.show()

NameError: name 'b2' is not defined

In [ ]:
b2.start_scope()

# Define all the parameters
input_current = oscillating_current()
N=100
stimulation_time=100*b2.ms
tau_m=10*b2.ms
v0=-50*b2.mV
eta=0*b2.mV
v_rest=15*b2.mV
R=50*b2.ohm
#beta=10e5*b2.Hz

# Define the differential equation 
eqs = """
dv/dt = ( -(v-v_rest) + R * input_current(t) ) / tau_m : volt
""" # add (unless refractory)

# Create the neuron group
neurons = b2.NeuronGroup(N, model=eqs, threshold='v>eta', reset='v=v0', method='linear') #or method='euler' 'exact

# Initialize the membrane potential at t=0 (uniformly distributed between v0 and eta)
neurons.v = 'rand()*(eta-v0)+v0'

# Monitor the membrane potential and firing times of neurons
potentials = b2.StateMonitor(neurons, 'v', record=0)
spikes = b2.SpikeMonitor(neurons)

# Run the simulation
b2.run(stimulation_time)

# Plot the results
plt.plot(spikes.t/b2.ms, spikes.i, '.k')
plt.xlabel('Time (ms)')
plt.ylabel('Neuron index')
plt.show()

plt.plot(potentials.t/b2.ms, np.squeeze(potentials.v))
plt.xlabel('Time (ms)')
plt.ylabel('v')
plt.show()


In [ ]:
def network_activity(spikes : list, N : int = N, dt : b2.ms = dt):
    """compute the network activity given the spikes of each neuron

    Args:
        spikes (list): the spikes of each neuron
        N (int, optional): the number of neurons. Defaults to N.
        dt (b2.ms, optional): the dt of the simulation. Defaults to dt.

    Returns:
        b2.TimedArray: the network activity at each timestep (starting from 0ms to Tf)
    """
    network_activity = []
    for k in range(int(Tf / dt) + 1):
        network_activity.append(sum([spikes[neuron](k*dt) for neuron in range(N)]))
    return b2.TimedArray(network_activity, dt=dt)

In [ ]:
# Plot the low-pass filtered activity of the population, using a time bin of τA = 1ms
tau_A = 1*b2.ms
low_pass_filtered_activity = []
for neuron in range(N):
    low_pass_filtered_activity.append(np.convolve(spikes[neuron].values, np.ones(int(tau_A/dt))/int(tau_A/dt), mode='same'))
low_pass_filtered_activity = np.array(low_pass_filtered_activity)
plt.plot(np.arange(len(low_pass_filtered_activity[0]))*dt/b2.ms, low_pass_filtered_activity[0], label='Low-pass filtered activity of the first neuron')
plt.title('Low-pass filtered activity of the first neuron')
plt.xlabel('Time (ms)')
plt.ylabel('Low-pass filtered activity')
plt.legend()
plt.show()